In [1]:
%matplotlib inline

In [2]:

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import signal, ndimage, misc

from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Dropout, GaussianNoise
from keras.layers import Convolution2D, Deconvolution2D, MaxPooling2D, UpSampling2D
from keras.layers import Convolution3D, UpSampling3D, MaxPooling3D
from keras.models import Model
from keras import regularizers
from keras import backend as K_backend
from keras import objectives
from keras.datasets import mnist
from keras.utils import np_utils

# Need this because otherwise the progbar freezes my Jupyter
from keras_tqdm import TQDMCallback, TQDMNotebookCallback


Using TensorFlow backend.


In [3]:
from matplotlib.pyplot import imshow
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [4]:
def nimshow(img):
    plt.imshow(img, interpolation='none')
    
def conv(img, kern, renorm=False):
    ift = signal.fftconvolve(img, kern, 'same')
    if renorm:
        ift /= np.amax(ift)
    return ift

def ricker2d(x, y, f=np.pi, n=0.5):
    r = (x**2 + y**2)**n
    return (1.0 - 2.0*(np.pi**2)*(f**2)*(r**2)) * np.exp(-(np.pi**2)*(f**2)*(r**2))

def gauss2d(x, y, f=1, sig=1, n=0.5):
    r = (x**2 + y**2)**n
    return np.exp(-((f*r)**2)/(.25*sig**2))

In [5]:
def plot_3d(image, threshold=-300, azim=45, elev=45):
    
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
#     p = p[:,:,::-1]
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    ax.azim = azim
    ax.elev = elev

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()
    
def splay(volume, rows=5, cols=5):
    fig = plt.figure(figsize=(10, 10))
    n = rows*cols
    for i in range(rows):
        for j in range(cols):
            fig.add_subplot(rows, cols, cols*i+j+1)
            plt.imshow(volume[cols*i+j])



In [6]:
x_train3d = np.load('/home/mike/py/kaggle/npdata/x_train_mnist3d_1k.npy')
x_train3d.shape

(1000, 28, 28, 28)

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
# x_train = np.reshape(x_train, (len(x_train), 1, 28, 28))
# x_test = np.reshape(x_test, (len(x_test), 1, 28, 28))
y_train_oh = np_utils.to_categorical(y_train)
y_test_oh = np_utils.to_categorical(y_test)
y_train_oh = y_train_oh[:len(x_train3d)]

In [8]:
x_train.shape

(60000, 28, 28)

In [9]:
# x_train3d_2x = sp.ndimage.zoom(x_train3d, (1,2,2,2))

In [10]:
def padcrop_vol(vol, newshape=[360, 360, 360]):
    vol2 = np.array(vol)
    shape = vol.shape
    z, y, x = shape
    mids = [d // 2 for d in shape]
    for dim in range(3):
        if shape[dim] < newshape[dim]:
            pad_amt = (newshape[dim] - shape[dim]) // 2
            parity = (shape[dim] & 1) ^ (newshape[dim] & 1)
            pad_tup = (pad_amt, pad_amt + parity) # 
            pad_list = [(0,0), (0,0), (0,0)]
            pad_list[dim] = pad_tup
            vol2 = np.pad(vol2, pad_list, mode='constant', constant_values=0)
        if shape[dim] > newshape[dim]:
            slc_amt = (shape[dim] - newshape[dim]) // 2
            parity = (shape[dim] & 1) ^ (newshape[dim] & 1)
            slc_tup = (slc_amt, shape[dim] - slc_amt - parity) # 
            null1, vol2, null2 = np.split(vol2, slc_tup, dim)

    return vol2

In [11]:
x_train3d_48 = [padcrop_vol(v, (48,48,48)) for v in x_train3d]
x_train3d_48 = np.array(x_train3d_48, dtype='float32')

In [12]:
n, x, y, z = x_train3d_48.shape
x_train3d_48 = x_train3d_48.reshape((n, x, y, z, 1))
x_train3d_48.shape

(1000, 48, 48, 48, 1)

In [13]:
n, x, y, z = x_train3d.shape
x_train3d = x_train3d.reshape((n, x, y, z, 1))
x_train3d.shape

(1000, 28, 28, 28, 1)

In [14]:
# n, x, y, z = x_train3d_2x.shape
# x_train3d_2x = x_train3d_2x.reshape((n, x, y, z, 1))
# x_train3d_2x.shape

In [15]:
class Convo3d(object):
    def __init__(self, input_shape=(28, 28, 28, 1), n_classes=10, n_filters=4):
        """
        5D tensor with shape: (samples, channels, conv_dim1, conv_dim2, conv_dim3) if dim_ordering='th' or 
        5D tensor with shape: (samples, conv_dim1, conv_dim2, conv_dim3, channels) if dim_ordering='tf'.
        """
#         input_img = Input(shape=(1, 28, 28, 28)) # th =(nChan, nFrames, xPix, yPix) or (nChan, z, x, y)
        input_img = Input(shape=input_shape) # th =(nChan, nFrames, xPix, yPix) or (nChan, z, x, y)
        img_chans = 1
        

        
        x = Convolution3D(n_filters, 3, 3, 3, activation='relu', border_mode='same')(input_img)
        x = MaxPooling3D((2, 2, 2), border_mode='same')(x)
        x = Convolution3D(n_filters*8, 3, 3, 3, activation='relu', border_mode='same')(x)
        x = MaxPooling3D((2, 2, 2), border_mode='same')(x)
        x = Convolution3D(n_filters*16, 3, 3, 3, activation='relu', border_mode='same')(x)
        self.z = MaxPooling3D((2, 2, 2), border_mode='same')(x)

        # at this point the representation is (4, 4, 4, 8) i.e. 512-dimensional
        
        flat = Flatten()(self.z)
        classer_base = Dense(n_classes, init='normal', activation='softmax', name='classer_output')(flat)

        x = Convolution3D(n_filters*16, 3, 3, 3, activation='relu', border_mode='same')(self.z)
        x = UpSampling3D((2, 2, 2))(x)
        x = Convolution3D(n_filters*8, 3, 3, 3, activation='relu', border_mode='same')(x)
        x = UpSampling3D((2, 2, 2))(x)
#         x = Convolution3D(16, 3, 3, 3, activation='relu', border_mode='valid')(x)
        x = UpSampling3D((2, 2, 2))(x)
        # smash the extra channels down
        x = Convolution3D(img_chans, 3, 3, 3, activation='sigmoid', border_mode='same')(x)
#         print(input_shape)
#         x = Dense(input_shape)(x) # fudge to fix output shape
        self.decoded = x
        self.autoencoder = Model(input_img, self.decoded)
        
        self.classifier = Model(input_img, classer_base)
        self.classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        

In [16]:
ae_base = Convo3d(x_train3d_48[0].shape)
ae = ae_base.autoencoder
ae.compile(optimizer='adadelta', loss='binary_crossentropy')

In [17]:
ae.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 48, 48, 48, 1) 0                                            
____________________________________________________________________________________________________
convolution3d_1 (Convolution3D)  (None, 48, 48, 48, 4) 112         input_1[0][0]                    
____________________________________________________________________________________________________
maxpooling3d_1 (MaxPooling3D)    (None, 24, 24, 24, 4) 0           convolution3d_1[0][0]            
____________________________________________________________________________________________________
convolution3d_2 (Convolution3D)  (None, 24, 24, 24, 32 3488        maxpooling3d_1[0][0]             
___________________________________________________________________________________________

The limit for a 3D 3-layer convonet VAE conks out on this laptop at batch_size=100, 236,500 parameters

In [18]:
batch_size=50
x_train_n = x_train3d_48
print('Tensor size: {}'.format(x_train_n.shape * batch_size))
ae.fit(x_train_n, x_train_n, shuffle=True,batch_size=batch_size,
               nb_epoch=1, 
               verbose=False, callbacks=[TQDMNotebookCallback()])

Tensor size: (1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1, 1000, 48, 48, 48, 1,

In [23]:
48**3 * 50

5529600

Works:
32^3 x 100 = 3276800
48^3 x 50 = 5529600

Does not work / questionable
48^3 x 100

In [21]:
ae_base.classifier.fit(x_train_n, y_train_oh, shuffle=True,batch_size=batch_size,
               nb_epoch=5, 
               verbose=False, callbacks=[TQDMNotebookCallback()])

In [24]:
ae_base.classifier.evaluate(x_train_n, y_train_oh)

1000/1000 [==============================] - 1s     

[0.14330089798569678, 0.94699999999999995]

In [ ]:
56**3 *100